In [ ]:
import numpy as np
from numpy import sin, cos, arccos
import numpy.linalg as la
import sympy as sp
from IPython.display import display

In [ ]:
'''
https://github-wiki-see.page/m/trip2eee/toydrone/wiki/3.2-Quaternion
'''

''' 
generate random unit vector (np.array (3,))
'''
def gen_random_uvec():
    v = np.random.rand(3)
    return v / la.norm(v)

''' 
convert angle axis representation (float, (np.array (3,))) into quaternion (np.array (4,))
'''
def angleaxis2quat(angle: float, axis: np.array, isDeg=True):
    assert len(axis) == 3
    
    eps = 1e-10
    if not abs(la.norm(axis) - 1) < eps:
        axis = axis / la.norm(axis)

    if isDeg:
        angle = np.deg2rad(angle)
        
    q = np.array([cos(angle/2), axis[0] * sin(angle/2), axis[1] * sin(angle/2), axis[2] * sin(angle/2)])

    return q

''' 
convert quaternion (np.array (4,)) into angle axis representation (float, (np.array (3,)))
'''
def quat2angleaxis(q: np.array, isDeg=True):
    assert len(q) == 4
    
    angle = 2 * arccos(q[0])
    axis = q[1:] / sin(angle / 2)
    
    if isDeg:
        angle = np.rad2deg(angle)
    
    return angle, axis

''' 
convert quaternion (np.array (4,)) into rotation matrix (np.array (3,3))
'''
def quat2Rot(q):
    R = np.array([[q[0]**2 + q[1]**2 - q[2]**2 - q[3]**2,             2*(q[1]*q[2] - q[0]*q[3]),              2*(q[0]*q[2] + q[1]*q[3])],
                  [            2*(q[1]*q[2] + q[0]*q[3]), q[0]**2 - q[1]**2 + q[2]**2 - q[3]**2,            2.0*(q[2]*q[3] - q[0]*q[1])],
                  [            2*(q[1]*q[3] - q[0]*q[2]),             2*(q[0]*q[1] + q[2]*q[3]),  q[0]**2 - q[1]**2 - q[2]**2 + q[3]**2]])
    return R

'''
get symbolic representation of quaternion matching
'''
def get_symbolic():
    v = sp.Matrix(['v_x', 'v_y', 'v_z'])
    q = sp.Matrix(['q_w', 'q_x', 'q_y', 'q_z'])

    R = quat2Rot(q)
    
    # define residual vector (\in R^{4}) and its jacobian w.r.t. quaternion elements
    F = R @ v
    J = F.jacobian(q)
    
    print("F: \n", print(F))
    print("J: \n", print(J))

    return F, J

'''
get function value at given (q, v)
'''
def get_func(q: np.array, v: np.array):
    R = quat2Rot(q)
    return R @ v

'''
get jacobian value at given (q, v)
'''
def get_jacb(q: np.array, v: np.array):
    J = np.array([[2*q[0]*v[0] + 2*q[2]*v[2] - 2*q[3]*v[1],  2*q[1]*v[0] + 2*q[2]*v[1] + 2*q[3]*v[2],  2*q[0]*v[2] + 2*q[1]*v[1] - 2*q[2]*v[0],  -2*q[0]*v[1] + 2*q[1]*v[2] - 2*q[3]*v[0]], 
                  [2*q[0]*v[1] - 2*q[1]*v[2] + 2*q[3]*v[0], -2*q[0]*v[2] - 2*q[1]*v[1] + 2*q[2]*v[0],  2*q[1]*v[0] + 2*q[2]*v[1] + 2*q[3]*v[2], 2*q[0]*v[0] + 2.0*q[2]*v[2] - 2*q[3]*v[1]], 
                  [2*q[0]*v[2] + 2*q[1]*v[1] - 2*q[2]*v[0],  2*q[0]*v[1] - 2*q[1]*v[2] + 2*q[3]*v[0], -2*q[0]*v[0] - 2*q[2]*v[2] + 2*q[3]*v[1],   2*q[1]*v[0] + 2*q[2]*v[1] + 2*q[3]*v[2]]])
    return J

In [ ]:
max_iter = 50
num_data = 200
eps=1e-10

# generate ground-truth and initial guess
q_gt = angleaxis2quat(30, gen_random_uvec())
q = angleaxis2quat(60, gen_random_uvec())
print(f"[true val] q: {q_gt}")
print(f"[iter 0] q: {q}")

# generate data
v_list = [None] * num_data
qv_list = [None] * num_data
for i in range(num_data):
    v = np.random.rand(3)                   # (3,) vector
    q_noise = angleaxis2quat(np.random.normal(loc=0, scale=10), gen_random_uvec())   # (4,) quaternion for rotation noise

    v_list[i] = v
    qv_list[i] = quat2Rot(q_noise) @ quat2Rot(q_gt) @ v

# run optimization
q_prev = angleaxis2quat(0, gen_random_uvec())
for i in range(1, max_iter+1):
    F = np.zeros(3*num_data)
    J = np.zeros((3*num_data, 4))

    # fill in jacobians and residuals
    for n in range(num_data):
        F[3*n: 3*(n+1)] = get_func(q, v_list[n]) - qv_list[n]
        J[3*n: 3*(n+1)] = get_jacb(q, v_list[n])

    # run levenberg-marquardt
    mu = np.exp(-i) * 1e-2
    d = - la.lstsq(J.T @ J + mu * np.eye(4), J.T @ F, rcond=None)[0]
    q = q + d
    print(la.norm(q))
    
    # apply termination condition
    if la.norm(q - q_prev) < eps:
        print(f"[iter {i}] q: {q} (improvement: {la.norm(q-q_prev)})")
        break
    else:
        print(f"[iter {i}] q: {q} (improvement: {la.norm(q-q_prev)})")
        q_prev = q
        continue

print("\n ####### SHOW RESULTS ####### \n")
angle_gt, axis_gt = quat2angleaxis(q_gt)
angle_et, axis_et = quat2angleaxis(q)
print(f"[truth    ] axis: {axis_gt}, angle: {angle_gt}")
print(f"[estimated] axis: {axis_et}, angle: {angle_et}")